In [2]:
import pandas as pd
import xgboost as xgb
import numpy as np

#
#import sklearn
#import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'xgboost'

In [3]:
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")
sample.head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'sample_submission.csv'

In [4]:
test_limpio = pd.read_csv("test_limpio_con_BOW_de_5000_y_Stemming.csv",encoding = "ISO-8859-1")
test_limpio.head()

FileNotFoundError: [Errno 2] No such file or directory: 'test_limpio_con_BOW_de_5000_y_Stemming.csv'

In [8]:
train_limpio = pd.read_csv("train_limpio_con_BOW_de_5000_y_Stemming.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,aa,ab,aba,abandon,abbott,abbswinston,abc,abcnew,abe,abia,...,money_related,tiene_fecha,has_year,tiene_hora,tiene_hora_2,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,70,2,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,40,1,0.1,0.4


In [9]:
train_y=train_limpio.target
train_y

0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
6967    0
6968    0
6969    0
6970    0
6971    0
6972    0
6973    0
6974    0
6975    0
6976    1
6977    1
6978    0
6979    1
6980    1
6981    1
6982    1
6983    1
6984    0
6985    1
6986    1
6987    1
6988    1
6989    1
6990    1
6991    1
6992    1
6993    1
6994    1
6995    1
6996    1
Name: target, Length: 6997, dtype: int64

In [10]:
train_x=train_limpio.drop([ 'target'], axis=1)
train_x.head()

,aa,ab,aba,abandon,abbott,abbswinston,abc,abcnew,abe,abia,...,money_related,tiene_fecha,has_year,tiene_hora,tiene_hora_2,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,70,2,0.00000,0.0000
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,40,1,0.10000,0.4000
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,130,2,-0.01875,0.3875
3,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,60,1,0.00000,0.0000
4,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,90,2,0.00000,0.0000


## Prueba basica

In [11]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)

In [12]:
#rf= RandomForestClassifier(n_estimators=50)
#xg_reg = xgb.XGBClassifier(objective ='reg:squarederror', 
#                colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)
xgc = xgb.XGBClassifier()
xgc.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
pred = xgc.predict(x_test)

In [14]:
f1_score(y_test.values, pred, average='macro')

0.7600889114681798

## Prueba con cross-validation

In [15]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [16]:
for train_index,test_index in kf.split(train_x):
    x_train,x_test = train_x.loc[train_index], train_x.loc[test_index]
    y_train,y_test = train_y[train_index],train_y[test_index]
    
    xgc = xgb.XGBClassifier()
    xgc.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = xgc.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = xgc.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))
    

Error de entrenamiento:  0.8658581663203013
Error de test:  0.7327779353332238
Error de entrenamiento:  0.8685363637546497
Error de test:  0.6949233502181154
Error de entrenamiento:  0.8660391032382713
Error de test:  0.7080942734677101
Error de entrenamiento:  0.8675324414171578
Error de test:  0.6875952183411588
Error de entrenamiento:  0.8571945043032508
Error de test:  0.7304431599229286


# tunning con griddsearch, masomenos andaba cuando tenia solo 1000 columnas, ahora imposible

In [21]:
params_grid={
    'n_estimators': [50,100,150,200],
    'learning_rate' : [0.3,0.45,0.6,0.75],
    'max_depth' : [3,6,9,12]
    
}

In [22]:
params_fixed={
    'objective':'binary:logistic'
}

In [23]:
bst_grid= GridSearchCV(
    estimator=xgb.XGBClassifier(**params_fixed),
    param_grid=params_grid,
    cv=5,
    scoring='accuracy'
)

In [24]:
bst_grid.fit(train_x, train_y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constrai...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=None, verbosity=None),
             iid='war

In [25]:
best_parameters = bst_grid.best_params_
print(best_parameters)

{'learning_rate': 0.3, 'max_depth': 12, 'n_estimators': 200}


Dio que {'learning_rate': 0.3, 'max_depth': 12, 'n_estimators': 200}

In [26]:
# Este fue el resultado con los mejores hyperparametros de antes, ahora supongo que van a ser otros

In [29]:
for train_index,test_index in kf.split(train_x):
    x_train,x_test = train_x.loc[train_index], train_x.loc[test_index]
    y_train,y_test = train_y[train_index], train_y[test_index]
    
    xgc = xgb.XGBClassifier(n_estimators=200 , learning_rate=0.3 , max_depth=12)
    xgc.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = xgc.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = xgc.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))
    

Error de entrenamiento:  0.9536573954539651
Error de test:  0.7413965527383963
Error de entrenamiento:  0.9571070772939287
Error de test:  0.7092006039938168
Error de entrenamiento:  0.9561912517676197
Error de test:  0.7278375320408567
Error de entrenamiento:  0.9586683993204866
Error de test:  0.6993058729304018
Error de entrenamiento:  0.9539169341081908
Error de test:  0.7436575055677621


In [119]:
#La realizo devuelta con mayor precision alrededor de los puntos seleccionados antes

In [31]:
params_grid={
    'n_estimators': [200,230,260],
    'learning_rate' : [0.1,0.2,0.3],
    'max_depth' : [12,15,18]
    
}

In [32]:
bst_grid= GridSearchCV(
    estimator=xgb.XGBClassifier(**params_fixed),
    param_grid=params_grid,
    cv=5,
    scoring='accuracy'
)

In [33]:
bst_grid.fit(train_x, train_y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constrai...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=None, verbosity=None),
             iid='war

In [34]:
best_parameters = bst_grid.best_params_
print(best_parameters)

{'learning_rate': 0.3, 'max_depth': 15, 'n_estimators': 230}


In [124]:
# Me da lo mismo, lo dejo aca

In [37]:
for train_index,test_index in kf.split(train_x):
    x_train,x_test = train_x.loc[train_index], train_x.loc[test_index]
    y_train,y_test = train_y[train_index], train_y[test_index]
    
    xgc = xgb.XGBClassifier(n_estimators=230,learning_rate=0.3, max_depth=15)
    xgc.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = xgc.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = xgc.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))

Error de entrenamiento:  0.968936509782346
Error de test:  0.7264957264957264
Error de entrenamiento:  0.973206909125544
Error de test:  0.7183382738382849
Error de entrenamiento:  0.9717313845258871
Error de test:  0.7205248898155894
Error de entrenamiento:  0.9715645456829164
Error de test:  0.6921892189218922
Error de entrenamiento:  0.9680655490619658
Error de test:  0.7438168043260881


In [41]:
# Entonces, mi modelo optimizado es:
xgc = xgb.XGBClassifier(n_estimators=230,learning_rate=0.3,max_depth=15)
xgc.fit(train_x, train_y)
    
#error de entrenamiento, es el unico que puedo calcular:
pred = xgc.predict(train_x)
print('Error de entrenamiento: ',f1_score(train_y, pred, average='macro'))

Error de entrenamiento:  0.9657802488557622


In [46]:
pred = xgc.predict(test_limpio)

In [47]:
sample.target=pred

In [48]:
sample.to_csv('prediccion_xgboost.csv', index=False)